<a href="https://colab.research.google.com/github/aegonwolf/GreatBarrierReefYolo/blob/main/CartPole_OpenAIGymPlayplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  #remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
import gym # openAi gym
from gym import envs
from gym.wrappers import Monitor
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [ ]:
np.max([5, 3])

5

In [ ]:
import gym
env = wrap_env(gym.make('CartPole-v1'))
env.action_space# View the number of actions available in the environment

Discrete(2)

In [ ]:
buckets = [10, 10, 30, 30]
obs_lb = env.observation_space.low
obs_lb[0] = -2.5
obs_lb[1] = -4
obs_lb[3] = -4
obs_lb[2] = -0.21

In [ ]:
rng = np.random.default_rng()

In [ ]:
def bucketize(observation):
  observation = (100*(observation - obs_lb)) / buckets
  return observation.astype(int)

In [ ]:
qtable = dict()
epsilon = 0.3

In [ ]:
obs = env.reset()
obs

array([0.01817212, 0.00318154, 0.02600345, 0.03800486])

In [ ]:
lr = 0.2

In [ ]:
def run_sim():
  observation = env.reset()
  observation = tuple(bucketize(observation))

  score = 0
  while True:
      # actions = []
      env.render()
      if observation in qtable:
        optimal_action = np.argmax(qtable[observation]['r'])
        action = rng.choice([optimal_action, 1 - optimal_action], p = [1 - epsilon, epsilon])
      else:
        action = rng.choice([0, 1])
        qtable[observation] = {'r': [0, 0], 'steps': [0, 0]}
      qtable[observation]['steps'][action] += 1
      current_reward = qtable[observation]['r'][action]
      new_observation, reward, done, info = env.step(action) 
      new_observation = tuple(bucketize(new_observation))
      if new_observation in qtable:
        max_reward = np.max(qtable[new_observation]['r'])
      else:
        max_reward = 0
      
      
      # actions.append((observation, action))
      score += reward         
      if done: 
        reward = -200
        qtable[observation]['r'][action] = (1 - lr)* current_reward + lr * (reward + 0.99 * max_reward - current_reward) #changed to TD
        break;    
      qtable[observation]['r'][action] = (1-lr) * current_reward + lr * (reward + 0.99 * max_reward - current_reward)
      observation = new_observation       
  env.close()
  # show_video()
  return score

In [ ]:
# env.close()

In [ ]:
avg = 0
for episode in range(50000):
  epsilon = epsilon*0.99995
  score = run_sim()
  avg += score
  if episode % 100 == 0:
    avg = avg / 100.0
    print(f'score avg of last 100 episodes @ {episode} is {avg}')
    avg = 0

score avg of last 100 episodes @ 0 is 0.11
score avg of last 100 episodes @ 100 is 22.22
score avg of last 100 episodes @ 200 is 28.12
score avg of last 100 episodes @ 300 is 28.74
score avg of last 100 episodes @ 400 is 27.88
score avg of last 100 episodes @ 500 is 31.8
score avg of last 100 episodes @ 600 is 32.85
score avg of last 100 episodes @ 700 is 33.54
score avg of last 100 episodes @ 800 is 35.42
score avg of last 100 episodes @ 900 is 38.2
score avg of last 100 episodes @ 1000 is 38.54
score avg of last 100 episodes @ 1100 is 40.56
score avg of last 100 episodes @ 1200 is 44.9
score avg of last 100 episodes @ 1300 is 39.68
score avg of last 100 episodes @ 1400 is 39.73
score avg of last 100 episodes @ 1500 is 44.79
score avg of last 100 episodes @ 1600 is 38.88
score avg of last 100 episodes @ 1700 is 42.64
score avg of last 100 episodes @ 1800 is 46.56
score avg of last 100 episodes @ 1900 is 48.84
score avg of last 100 episodes @ 2000 is 47.4
score avg of last 100 episodes

In [ ]:
epsilon = 0.1

In [ ]:
avg = 0
for episode in range(10000):
  epsilon = epsilon*0.99995
  score = run_sim()
  avg += score
  if episode % 100 == 0:
    avg = avg / 100.0
    print(f'score avg of last 100 episodes @ {episode} is {avg}')
    avg = 0

score avg of last 100 episodes @ 0 is 0.37
score avg of last 100 episodes @ 100 is 162.39
score avg of last 100 episodes @ 200 is 133.53
score avg of last 100 episodes @ 300 is 157.2
score avg of last 100 episodes @ 400 is 136.58
score avg of last 100 episodes @ 500 is 117.19
score avg of last 100 episodes @ 600 is 154.39
score avg of last 100 episodes @ 700 is 165.4
score avg of last 100 episodes @ 800 is 154.61
score avg of last 100 episodes @ 900 is 153.79
score avg of last 100 episodes @ 1000 is 153.54
score avg of last 100 episodes @ 1100 is 146.08
score avg of last 100 episodes @ 1200 is 153.14
score avg of last 100 episodes @ 1300 is 157.78
score avg of last 100 episodes @ 1400 is 154.65
score avg of last 100 episodes @ 1500 is 143.74
score avg of last 100 episodes @ 1600 is 149.13
score avg of last 100 episodes @ 1700 is 166.74
score avg of last 100 episodes @ 1800 is 145.53
score avg of last 100 episodes @ 1900 is 157.55
score avg of last 100 episodes @ 2000 is 139.82
score av

In [ ]:
epsilon

0.02462396049024598

In [ ]:
def run_optimal():
  observation = env.reset() # Reset the environment
  observation = tuple(bucketize(observation))
  score = 0
  while True:
      env.render()
      if observation in qtable:
        optimal_action = np.argmax(qtable[observation]['r'])
        action = optimal_action
      else:
        print('new observation!')
        action = rng.choice([0, 1])
        qtable[observation] = {'r': [0, 0], 'steps': [0, 0]}
      qtable[observation]['steps'][action] += 1
      current_reward = qtable[observation]['r'][action]
      new_observation, reward, done, info = env.step(action) 
      observation = tuple(bucketize(new_observation))
      #avg reward
      # qtable[observation]['r'][action] += current_reward + ((reward - current_reward) / qtable[observation]['steps'][action]) + 0.1*score
      # print(observation)
      # observation = tuple(np.around(o, 2) for o in new_observation)
      score += reward         
      if done: 
        break;             
  print(score)
  show_video()
  env.close()

In [ ]:
run_optimal()

226.0


In [ ]:
show_video()